In [ ]:
#Loading Libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pylab
import math
from math import log

import pylab as plot
import numpy as np
import seaborn as sns

import sklearn
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import datasets
from sklearn import linear_model
from sklearn import ensemble

In [ ]:
#Reading the file
listings = pd.read_csv('listings.csv')

In [ ]:
listings.columns.tolist()

In [ ]:
#EDA
plt.figure(figsize=(18,10))
join_dates = pd.to_datetime(listings['host_since']).value_counts().resample('D').mean().fillna(0)
join_dates.plot()
plt.xlabel('year')
plt.ylabel('number of hosts')

In [ ]:
#Cleaning the price
listings['price'] = (listings['price'].str.replace(r'[^-+\d.]', '').astype(float))

In [ ]:
#Selecting only non zero values
listings = listings[listings['accommodates'] != 0]
listings = listings[listings['bedrooms'] != 0]
listings = listings[listings['beds'] != 0]
listings = listings[listings['price'] != 0.00]

In [ ]:
Transformed_df = listings[['price',
           'room_type',
           'accommodates',
           'bathrooms',
           'bedrooms',
           'beds',
           'review_scores_rating',
           'instant_bookable',
           'cancellation_policy',
           'amenities']]

In [ ]:
#Converting the string records into seperate elements and counting
cancel_policy = pd.get_dummies(Transformed_df.cancellation_policy).astype(int)
room_type = pd.get_dummies(Transformed_df.room_type).astype(int)
instant_booking = pd.get_dummies(Transformed_df.instant_bookable, prefix = 'instant_booking').astype(int)

In [ ]:
#Removing where instant booking is not availible
instant_booking = instant_booking.drop('instant_booking_f', axis = 1)

In [ ]:
# we drop the original columns and replace them with new columns
Transformed_df = Transformed_df.drop(['cancellation_policy', 'instant_bookable', 'room_type'], axis = 1)
Transformed_df = pd.concat((Transformed_df, cancel_policy, instant_booking, room_type), axis = 1)

In [ ]:
# splitting the amenities list to draw out how many amenities each listing has

amenities_list = []

for element in Transformed_df.amenities:
    element = element[1:]
    element = element[:-1]
    x = element.split()
    amenities_list.append(len(x))

Transformed_df.amenities = amenities_list

In [ ]:
Transformed_df.head()

In [ ]:
Transformed_df= Transformed_df.dropna()

In [ ]:
# mean of prices
mean = np.mean(Transformed_df.price)

# standard deviation
std = np.std(Transformed_df.price)

print("mean : " , mean)
print ("sd   : " ,std)

In [ ]:
# splitting the training and test sets with a 60% and 40% size of original
split_data = Transformed_df.drop(['price'], axis = 1)
split_data.shape

In [ ]:
X = split_data
y = Transformed_df.price

In [ ]:
Transformed_df.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13,test_size=.2)

In [ ]:
#Logistic Regression
logistic = LogisticRegression()
logistic.fit(X_train, y_train)

print(format(logistic.score(X_test, y_test)))

In [ ]:
#Linear Regression
from sklearn.linear_model import LinearRegression
linear_reg = linear_model.LinearRegression()
linear_reg.fit(X_train, y_train)
linear_reg_error = metrics.median_absolute_error(y_test, linear_reg.predict(X_test))
print ("Error in Linear Regression: " + str(linear_reg_error))


In [ ]:
# Ridge Regression
ridge = linear_model.Ridge()
ridge.fit(X_train, y_train)
ridge_error = metrics.median_absolute_error(y_test, ridge.predict(X_test))
print ("Error in Ridge: " + str(ridge_error))

In [ ]:
# Support vector Machine
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
svm_error = metrics.median_absolute_error(y_test, svm.predict(X_test))

#print('Accuracy of SVM classifier: {:.2f}'.format(svm.score(X_test, y_test)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier: {:.2f}'.format(decision.score(X_test, y_test)))

In [ ]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
print('Accuracy of RF classifier: {:.2f}'.format(classifier.score(X_test, y_test)))


In [ ]:
y_pred=decision.predict(X_test) # This is changed every time for all the models

In [ ]:
print("Mean squared error: %.2f" % np.mean((decision.predict(X_test) - y_test) ** 2))# This is changed every time for all the models

In [ ]:
#Confusion Matrix
print(pd.crosstab(y_test,y_pred,colnames=["predicted"],rownames=["Actual"]))
